In [2]:
import os
import requests
from typing import List, Dict

SERPER_API_KEY = "2931bbdd6b21da755b6bf2794ba2c33134e8a2bb"
if not SERPER_API_KEY:
    raise RuntimeError(
        "Missing SERPER_API_KEY environment variable. "
        "Obtain one from https://serper.dev/ and set it before running."
    )

# Serper.dev search endpoint
_SERPER_SEARCH_URL = "https://google.serper.dev/search"

def search_serper(query: str, num_results: int = 5) -> List[Dict[str, str]]:
    """
    Query Serper.dev and return up to `num_results` organic search hits.
    Each hit is a dict: { "title": str, "link": str, "snippet": str }.
    Raises HTTPError on non-2xx responses.
    """
    headers = {
        "X-API-KEY": SERPER_API_KEY,
        "Content-Type": "application/json"
    }
    payload = {
        "q": query,
        "num": num_results,
    }

    resp = requests.post(_SERPER_SEARCH_URL, headers=headers, json=payload, timeout=5)
    resp.raise_for_status()
    data = resp.json()

    results = []
    for item in data.get("organic", [])[:num_results]:
        title = item.get("title", "").strip()
        link = item.get("link", "").strip()
        snippet = item.get("snippet", "").strip()
        results.append({"title": title, "link": link, "snippet": snippet})
    return results

In [ ]:
from typing import List, Dict

def scrape_full_text(url: str, max_chars: int = 2000) -> str:
    """
    Try Newspaper3k first; on failure, fall back to BeautifulSoup.
    Returns up to `max_chars` of raw text.
    """
    try:
        art = Article(url)
        art.download()
        art.parse()
        text = art.text
        if text and len(text) > 50:
            return text[:max_chars]
    except Exception:
        pass

    try:
        resp = requests.get(url, timeout=5)
        resp.raise_for_status()
        soup = BeautifulSoup(resp.text, "html.parser")
        paras = soup.find_all("p")
        joined = " ".join(p.get_text() for p in paras)
        return joined[:max_chars]
    except Exception:
        return ""

def retrieve_evidence_for_claim(claim: str, num_results: int = 5) -> List[Dict[str, str]]:
    """
    For a given `claim` string, run Serper search and scrape each result.

    Returns a list of dicts:
      {
        "url": str,
        "title": str,
        "snippet": str,
        "raw_text": str
      }
    """
    hits = search_serper(claim, num_results)
    evidence = []

    for item in hits:
        url    = item["link"]
        title  = item["title"]
        snippet= item["snippet"]
        raw    = scrape_full_text(url)

        evidence.append({
            "url": url,
            "title": title,
            "snippet": snippet,
            "raw_text": raw
        })

    return evidence


In [5]:
pip install langchain langchain-community transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 97.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 71.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 63.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.1/438.1 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3

In [1]:
pip install sentence-transformers faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 27.3 MB/s eta 0:00:00


In [3]:
from langchain.embeddings import HuggingFaceEmbeddings


def load_embeddings_model(model_name="sentence-transformers/all-MiniLM-L6-v2"):
    """
    Loads a HuggingFace sentence embedding model for use in LangChain vectorstore.
    """
    print(f"Loading embedding model: {model_name}")
    return HuggingFaceEmbeddings(model_name=model_name)


In [10]:
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from transformers import pipeline


#Load LLM
def load_llm(model_name="Qwen/Qwen1.5-0.5B", device=0):
    print(f"Loading model: {model_name}")
    hf_pipeline = pipeline("text-generation", model=model_name, device=device, max_new_tokens=512)
    return HuggingFacePipeline(pipeline=hf_pipeline)


#Convert search results to LangChain Document objects
def build_documents(evidence_list):
    docs = []
    for entry in evidence_list:
        text = entry.get("raw_text") or entry.get("snippet")
        if text and text.strip():
            content = f"Title: {entry['title']}\nURL: {entry['url']}\n\n{text.strip()}"
            docs.append(Document(page_content=content))
    print(f"📄 Built {len(docs)} documents from evidence.")
    return docs


#Build vectorstore from documents
def create_vectorstore(documents):
    embeddings = load_embeddings_model()
    return FAISS.from_documents(documents, embeddings)


#Assemble RAG chain
def build_rag_chain(llm, vectorstore):
    retriever = vectorstore.as_retriever()
    return RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever,
        chain_type="stuff"
    )

#Full pipeline
def answer_claim_with_rag(claim):
    print(f"🔍 Searching for claim: {claim}")
    evidence_list = retrieve_evidence_for_claim(claim, num_results=5)

    if not evidence_list:
        return "❌ No relevant search results found."

    documents = build_documents(evidence_list)
    if not documents:
        return "⚠️ Evidence was found, but no usable content was extracted."

    vectorstore = create_vectorstore(documents)
    llm = load_llm()
    rag_chain = build_rag_chain(llm, vectorstore)

    print("🤖 Running LLM with RAG...")
    return rag_chain.run(claim)

# 🔹 Command-line entry point
if __name__ == "__main__":
    claim = input("Enter a claim to verify:\n")
    response = answer_claim_with_rag(claim)
    print("\n📢 Verdict:\n", response)


Enter a claim to verify:
vaccine cause autism
🔍 Searching for claim: vaccine cause autism
📄 Built 5 documents from evidence.
🔧 Loading embedding model: sentence-transformers/all-MiniLM-L6-v2
Loading model: Qwen/Qwen1.5-0.5B


config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.24G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

Device set to use cpu
<ipython-input-10-1468928201>:13: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  return HuggingFacePipeline(pipeline=hf_pipeline)
<ipython-input-10-1468928201>:61: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  return rag_chain.run(claim)


🤖 Running LLM with RAG...

📢 Verdict:
 Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Title: Vaccination as a cause of autism—myths and controversies - PMC
URL: https://pmc.ncbi.nlm.nih.gov/articles/PMC5789217/

Myths that vaccines or mercury are associated with autism have been amplified by misguided scientists; frustrated, but effective parent groups; and politicians.

Title: Vaccines and Autism | Children's Hospital of Philadelphia
URL: https://www.chop.edu/vaccine-education-center/vaccine-safety/vaccines-and-other-conditions/autism

Two studies have been cited by those claiming that the MMR vaccine causes autism. Both studies are critically flawed.

Title: Do vaccines cause autism? - Autism Speaks
URL: https://www.autismspeaks.org/do-vaccines-cause-autism

The result of this research is clear: vaccines do not cause autism. Additionally, vaccination can protect childr